In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import sklearn as sk
import pickle

import tensorflow as tf

2025-09-05 02:37:17.792130: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-05 02:37:18.047722: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-05 02:37:18.427872: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757039838.661891 2459007 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757039838.705491 2459007 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757039839.042489 2459007 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
# Load the dataset without headers (so every row is treated as data)
df = pd.read_csv("diabetes.csv", header=None)

# Step 1: Save the first row (metadata)
metadata = df.iloc[0].tolist()

print('before dropping meta', df.head())
# Step 2: Drop the first row
df = df.drop(0)
print('after dropping meta', df.head())

# Step 3: Make the next row the header
df.columns = df.iloc[0]   # assign row 1 as header
print('after setting colums header', df.head())
df = df.drop(1)
print('after dropping initial column row', df.head())

df = df.reset_index(drop=True)
print('after resetting indices', df.head())


before dropping meta              0        1              2              3        4     5  \
0          768        8            NaN            NaN      NaN   NaN   
1  Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI   
2            6      148             72             35        0  33.6   
3            1       85             66             29        0  26.6   
4            8      183             64              0        0  23.3   

                          6    7        8  
0                       NaN  NaN      NaN  
1  DiabetesPedigreeFunction  Age  Outcome  
2                     0.627   50        1  
3                     0.351   31        0  
4                     0.672   32        1  
after dropping meta              0        1              2              3        4     5  \
1  Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI   
2            6      148             72             35        0  33.6   
3            1       85             66        

In [9]:
columns = df.columns.tolist()
x_columns = columns[:-1]
y_columns = columns[-1]

In [63]:
df.dropna(inplace=True)

We dont have categorical data, so no need for encoding

In [57]:
#changes object types to numeric
le = sk.preprocessing.LabelEncoder()
for col in df.select_dtypes(include=['object']).columns:
    df[col] = le.fit_transform(df[col])

print(df.head())

1  Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin  BMI  \
0           13       49             30             26        0  123   
1            1      121             27             20        0   62   
2           15       84             25              0        0   30   
3            1      125             27             14      182   77   
4            0       38             12             26       40  209   

1  DiabetesPedigreeFunction  Age  Outcome  
0                       350   29        1  
1                       196   10        0  
2                       368   11        1  
3                        53    0        0  
4                       514   12        1  


In [37]:
Y = df[y_columns]
X = df.drop([y_columns], axis=1)

print('x head', X.head())
print('y head', Y.head())

x head 1  Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin  BMI  \
0           13       49             30             26        0  123   
1            1      121             27             20        0   62   
2           15       84             25              0        0   30   
3            1      125             27             14      182   77   
4            0       38             12             26       40  209   

1  DiabetesPedigreeFunction  Age  
0                       350   29  
1                       196   10  
2                       368   11  
3                        53    0  
4                       514   12  
y head 0    1
1    0
2    1
3    0
4    1
Name: Outcome, dtype: int64


In [38]:
X_train, x_test, Y_train, y_test = sk.model_selection.train_test_split(X, Y, test_size=0.2, shuffle=True, random_state=42)

In [39]:
#Normalize the numeric data

scaler = sk.preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)  # learn scaling from training set
x_test = scaler.transform(x_test)

In [42]:
print(' x head', X_train[5])
print('y head', Y_train[5])
print(len(Y_train))

 x head [-1.14427168 -0.54126755  0.08508077 -0.82156247 -0.694906   -0.9452733
  0.30720762 -0.95259703]
y head 0
614


In [43]:
print(X_train[0].shape, x_test.shape)

(8,) (154, 8)


In [66]:
model = tf.keras.models.Sequential([
  tf.keras.Input(shape=X_train[0].shape),
  tf.keras.layers.Dense(16, activation='relu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [67]:
model.compile(optimizer='adam',
  loss='binary_crossentropy',
  metrics=['accuracy'])

In [68]:
model.fit(X_train, Y_train, epochs=5, batch_size=32, validation_split=0.2)

Epoch 1/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.5438 - loss: 0.6962 - val_accuracy: 0.5772 - val_loss: 0.6761
Epoch 2/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5886 - loss: 0.6657 - val_accuracy: 0.6260 - val_loss: 0.6578
Epoch 3/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6130 - loss: 0.6535 - val_accuracy: 0.6260 - val_loss: 0.6461
Epoch 4/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6538 - loss: 0.6226 - val_accuracy: 0.6260 - val_loss: 0.6353
Epoch 5/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6762 - loss: 0.6124 - val_accuracy: 0.6260 - val_loss: 0.6255
